In [ ]:
from platypus import NSGAII, Problem, Real, Integer, Solution
import matplotlib.pyplot as plt

# Define the problem
def evaluate(solution):
    IP = solution.variables[0]
    LH = solution.variables[1]
    NW = int(solution.variables[2])
    BT = solution.variables[3]

    # Tensile Strength
    TS = (
        -0.830244 + 0.356997 * IP + 104.608 * LH + 11.9154 * NW - 0.184526 * BT
        - 0.00150299 * IP ** 2 + 20.6051 * LH ** 2 - 0.901996 * NW ** 2 + 0.00294304 * BT ** 2
        - 0.602183 * IP * LH - 0.00695 * IP * NW + 0.000542375 * IP * BT
        - 8.912 * LH * NW - 0.299033 * LH * BT - 0.01849 * NW * BT
    )

    # Mass
    M = (
        0.679909 + 0.00999339 * IP + 0.654401 * LH + 0.209838 * NW - 0.00244999 * BT
        - 2.72106e-06 * IP ** 2 + 7.62339 * LH ** 2 - 0.00814842 * NW ** 2 + 3.59158e-05 * BT ** 2
        - 0.0176952 * IP * LH - 8.19231e-04 * IP * NW - 9.54812e-06 * IP * BT
        - 0.283995 * LH * NW - 0.00441283 * LH * BT - 1.41288e-04 * NW * BT
    )

    # Platypus minimizes both objectives → minimize -TS to maximize TS
    solution.objectives[:] = [-TS, M]

# Define the optimization problem
problem = Problem(4, 2)  # 4 variables, 2 objectives
problem.types[:] = [Real(40, 80), Real(0.1, 0.25), Integer(2, 4), Real(45, 65)]
problem.function = evaluate

# Run NSGA-II
algorithm = NSGAII(problem)
algorithm.run(2000)  # Number of evaluations

# Extract results
ts_vals = [-s.objectives[0] for s in algorithm.result]
mass_vals = [s.objectives[1] for s in algorithm.result]

# Plot the Pareto front
plt.figure(figsize=(8, 6))
plt.scatter(mass_vals, ts_vals, c='blue')
plt.xlabel("Mass (g)")
plt.ylabel("Tensile Strength (MPa)")
plt.title("Pareto Front (NSGA-II using Platypus)")
plt.grid(True)
plt.show()

# Print top 5 solutions
print("\nTop 5 trade-off solutions:")
for i, s in enumerate(algorithm.result[:5]):
    IP, LH, NW, BT = s.variables
    print(f"{i+1}) IP={IP:.2f}, LH={LH:.3f}, NW={int(NW)}, BT={BT:.1f} | TS={-s.objectives[0]:.2f}, Mass={s.objectives[1]:.2f}")
